In [21]:
import numpy as np
import matplotlib.pyplot as plt
import trimesh

# Fonction pour normaliser un vecteur
def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm != 0 else v

# Fonction pour calculer le produit scalaire
def dot(v1, v2):
    return np.dot(v1, v2)

# Fonction pour calculer la réflexion d'un vecteur par rapport à un vecteur normal
def reflect(v, n):
    return v - 2 * dot(v, n) * n

# Fonction pour charger un modèle OBJ avec trimesh
def load_obj(filename):
    mesh = trimesh.load_mesh(filename)
    triangles = []
    for face in mesh.faces:
        v0, v1, v2 = [mesh.vertices[i] for i in face]
        triangles.append((v0, v1, v2))
    return triangles, mesh

# Classe pour les triangles
class Triangle:
    def __init__(self, v0, v1, v2):
        self.v0 = np.array(v0)
        self.v1 = np.array(v1)
        self.v2 = np.array(v2)
        self.normal = normalize(np.cross(v1 - v0, v2 - v0))

    def intersect(self, ray_origin, ray_dir):
        # Algorithme de Möller-Trumbore pour l'intersection de rayons avec un triangle
        epsilon = 1e-8
        edge1 = self.v1 - self.v0
        edge2 = self.v2 - self.v0
        h = np.cross(ray_dir, edge2)
        a = dot(edge1, h)
        if -epsilon < a < epsilon:
            return None
        f = 1.0 / a
        s = ray_origin - self.v0
        u = f * dot(s, h)
        if u < 0.0 or u > 1.0:
            return None
        q = np.cross(s, edge1)
        v = f * dot(ray_dir, q)
        if v < 0.0 or u + v > 1.0:
            return None
        t = f * dot(edge2, q)
        if t > epsilon:
            return t
        else:
            return None

# Fonction de rendu principale
def render(width, height, samples, model_file):
    img = np.zeros((height, width, 3))  # Réinitialiser l'image ici
    for y in range(height):
        for x in range(width):
            color = np.zeros(3)
            for s in range(samples):
                u = (2 * (x + np.random.rand()) / width - 1) * aspect_ratio * np.tan(fov / 2)
                v = (1 - 2 * (y + np.random.rand()) / height) * np.tan(fov / 2)
                ray_dir = normalize(np.array([u, v, -1]))

                t_min = float('inf')
                closest_triangle = None
                for triangle in triangles:
                    tri = Triangle(*triangle)
                    t = tri.intersect(camera_pos, ray_dir)
                    if t and t < t_min:
                        t_min = t
                        closest_triangle = tri

                if closest_triangle:
                    normal = closest_triangle.normal
                    diffuse = max(dot(normal, light_dir), 0)
                    color += np.array([1, 0, 0]) * diffuse

            img[y, x] = np.clip(color / samples, 0, 1)

    return img

# Paramètres de rendu
width, height = 100, 100
samples = 1
model_file = "src/sphere.obj"
camera_pos = np.array([0, 0, 3])  # Changer la position de la caméra
light_dir = np.array([1, 5, 10])
aspect_ratio = width / height
fov = np.pi / 3  # 60 degrés
triangles, mesh = load_obj(model_file)

# Rendu de l'image
image = render(width, height, samples, model_file)

# Affichage de l'image
plt.imshow(image)
plt.axis('off')
plt.show()


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [17]:
import plotly.graph_objects as go

model_file = "src/sphere.obj"
triangles, mesh = load_obj(model_file)

# Préparer les données pour Plotly
x, y, z = [], [], []
for triangle in triangles:
    for vertex in triangle:
        x.append(vertex[0])
        y.append(vertex[1])
        z.append(vertex[2])
print(x)
camera_pos = np.array([0, 0, 50])

fig = go.Figure()

# Ajouter le maillage
fig.add_trace(go.Scatter3d(
    x=x, 
    y=y, 
    z=z,
    name='Mesh'
))

# Ajouter la position de la caméra
fig.add_trace(go.Scatter3d(
    x=[camera_pos[0]], 
    y=[camera_pos[1]], 
    z=[camera_pos[2]],
    mode='markers+text',
    marker=dict(size=10, color='red'),
    text=['Camera Position'],
    textposition='top center',
    name='Camera'
))

light_dir = normalize(np.array([1, 90, 90]))
fig.add_trace(go.Scatter3d(
    x=[light_dir[0]], 
    y=[light_dir[1]], 
    z=[light_dir[2]],
    mode='markers+text',
    marker=dict(size=10, color='red'),
    text=['Light Direction'],
    textposition='top center',
    name='Light'
))

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

fig.show()

[0.148778, 0.126558, 0.25, 0.25, 0.293893, 0.148778, 0.126558, 0.09195, 0.181636, 0.181636, 0.25, 0.126558, 0.09195, 0.048341, 0.095492, 0.095492, 0.181636, 0.09195, 0.048341, 0.0, 0.0, 0.0, 0.095492, 0.048341, 0.0, -0.048341, -0.095492, -0.095492, 0.0, 0.0, -0.048341, -0.09195, -0.181636, -0.181636, -0.095492, -0.048341, -0.09195, -0.126558, -0.25, -0.25, -0.181636, -0.09195, -0.126558, -0.148778, -0.293893, -0.293893, -0.25, -0.126558, -0.148778, -0.156435, -0.309017, -0.309017, -0.293893, -0.148778, -0.156435, -0.148778, -0.293893, -0.293893, -0.309017, -0.156435, -0.148778, -0.126558, -0.25, -0.25, -0.293893, -0.148778, -0.126558, -0.09195, -0.181636, -0.181636, -0.25, -0.126558, -0.09195, -0.048341, -0.095492, -0.095492, -0.181636, -0.09195, -0.048341, -0.0, -0.0, -0.0, -0.095492, -0.048341, -0.0, 0.048341, 0.095491, 0.095491, -0.0, -0.0, 0.048341, 0.09195, 0.181636, 0.181636, 0.095491, 0.048341, 0.09195, 0.126558, 0.25, 0.25, 0.181636, 0.09195, 0.126558, 0.148778, 0.293893, 0.293